<a href="https://colab.research.google.com/github/iam-Dylan/automated-essay-scoring/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Learning Agency Lab - Automated Essay Scoring 2.0

- Môn học: Phân tích dữ liệu thông minh
- Nhóm: 10

# **THỬ NGHIỆM TRÊN MÔ HÌNH TỰ XÂY DỰNG**

##  **A. Tiền xử lý dữ liệu**


### **1. Import các thư viện cần thiết**

- Cài đặt thư viện cần thiết.

In [10]:
!pip install pyspellchecker

In [11]:
import pandas as pd
import polars as pl
import numpy as np
import re
import string

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import cohen_kappa_score, accuracy_score
import lightgbm as lgb

import nltk
from nltk.corpus import stopwords
from nltk.tokenize.treebank import TreebankWordDetokenizer
from spellchecker import SpellChecker
import warnings
warnings.filterwarnings('ignore')

### **2. Đọc dữ liệu**

In [12]:
# URL tải xuống trực tiếp của tệp CSV trên Google Drive
TRAIN_ID = '1hUhF4f-gGTixo_-b-ytez01_swNBslIG'
url = f"https://drive.google.com/uc?export=download&id={TRAIN_ID}"
# Đọc tệp CSV từ URL
try:
    train = pd.read_csv(url)
    display(train.head())
except Exception as e:
    print(f"Đã xảy ra lỗi: {e}")

,essay_id,full_text,score
0,000d118,Many people have car where they live. The thin...,3
1,000fe60,I am a scientist at NASA that is discussing th...,3
2,001ab80,People always wish they had the same technolog...,4
3,001bdc0,"We all heard about Venus, the planet without a...",4
4,002ba53,"Dear, State Senator\n\nThis is a letter to arg...",3


In [13]:
TEST_ID = '1kJa0kIeP0RpAFFcKa1QpFP7o4xtpxjet'
url = f"https://drive.google.com/uc?export=download&id={TEST_ID}"
# Đọc tệp CSV từ URL
try:
    test = pd.read_csv(url)
    display(test.head())
except Exception as e:
    print(f"Đã xảy ra lỗi: {e}")

,essay_id,full_text
0,000d118,Many people have car where they live. The thin...
1,000fe60,I am a scientist at NASA that is discussing th...
2,001ab80,People always wish they had the same technolog...


### **3. Tiền xử lý dữ liệu**

Cần **làm sạch văn bản**, nhằm chuẩn hóa và loại bỏ những thành phần không cần thiết trước khi tiến hành các bước xử lý tiếp theo.

- Văn bản được chuyển đổi toàn bộ về **chữ thường** để đảm bảo tính nhất quán và tránh phân biệt giữa chữ hoa và chữ thường.
- Các **thẻ HTML**, thẻ tên người dùng (bắt đầu bằng @), **hashtag** (bắt đầu bằng #), và đường dẫn **URL** đều được loại bỏ để giữ lại nội dung văn bản thực sự.
- Các **ký tự đặc biệt** và các **số** trong văn bản, thường không mang lại giá trị ngữ nghĩa, cũng được loại bỏ.
- Các **dấu câu liên tiếp** được xử lý và thay thế bằng một ký tự duy nhất.
- Các **từ viết tắt** được mở rộng thành dạng đầy đủ để đảm bảo tính nhất quán. Tham khảo từ: [Expand Contractions](https://www.kaggle.com/code/xianhellg/more-feature-engineering-feature-selection-0-817?scriptVersionId=173223907&cellId=11)

In [14]:
def expand_contractions(text):
    # Mở rộng các từ viết tắt.
    contractions_dict = {
    "ain't": "am not", "aren't": "are not", "can't": "cannot", "can't've": "cannot have", "'cause": "because", "could've": "could have",
    "couldn't": "could not", "couldn't've": "could not have", "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not",
    "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have","he'll": "he will", "he'll've": "he will have", "he's": "he is",
    "how'd": "how did","how'd'y": "how do you","how'll": "how will","how's": "how is",
    "I'd": "I would",
    "I'd've": "I would have","I'll": "I will","I'll've": "I will have","I'm": "I am","I've": "I have","isn't": "is not",
    "it'd": "it had",
    "it'd've": "it would have","it'll": "it will","it'll've": "it will have","it's": "it is",
    "let's": "let us","ma'am": "madam","mayn't": "may not","might've": "might have","mightn't": "might not","mightn't've": "might not have",
    "must've": "must have","mustn't": "must not","mustn't've": "must not have",
    "needn't": "need not","needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not","oughtn't've": "ought not have",
    "shan't": "shall not","sha'n't": "shall not","shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have","she'll": "she will","she'll've": "she will have","she's": "she is",
    "should've": "should have","shouldn't": "should not","shouldn't've": "should not have",
    "so've": "so have","so's": "so is",
    "that'd": "that would",
    "that'd've": "that would have","that's": "that is",
    "there'd": "there had",
    "there'd've": "there would have","there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have","they'll": "they will","they'll've": "they will have","they're": "they are","they've": "they have",
    "to've": "to have","wasn't": "was not","weren't": "were not",
    "we'd": "we had",
    "we'd've": "we would have","we'll": "we will","we'll've": "we will have","we're": "we are","we've": "we have",
    "what'll": "what will","what'll've": "what will have","what're": "what are","what's": "what is","what've": "what have",
    "when's": "when is","when've": "when have",
    "where'd": "where did","where's": "where is","where've": "where have",
    "who'll": "who will","who'll've": "who will have","who's": "who is","who've": "who have","why's": "why is","why've": "why have",
    "will've": "will have","won't": "will not","won't've": "will not have",
    "would've": "would have","wouldn't": "would not","wouldn't've": "would not have",
    "y'all": "you all","y'alls": "you alls","y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are",
    "y'all've": "you all have","you'd": "you had","you'd've": "you would have","you'll": "you you will","you'll've": "you you will have",
    "you're": "you are",  "you've": "you have"
    }
    contractions_re = re.compile('(%s)' % '|'.join(contractions_dict.keys()))

    return contractions_re.sub(lambda match: contractions_dict[match.group(0)], text)

def clean_text(text):
    # Chuyển chữ viết hoa thành chữ thường
    text = text.lower()

    # Xóa các thẻ HTML
    text = re.compile(r'<.*?>').sub(r'', text)

    # Xóa các tag tên (mention)
    text = re.sub(r'@\w+\s*', '', text)

    # Xóa hashtag (dấu #)
    text = re.sub(r'#\w+', '', text)

    # Xóa các liên kết URL
    text = re.sub(r'http\S+|www\S+', '', text)

    # Xóa các ký tự không mong muốn như \xa0
    text = text.replace(u'\xa0', ' ')

    # Xóa chữ số
    text = re.sub(r'\d+', '', text)

    # Mở rộng các từ viết tắt
    text = expand_contractions(text)

    # Thay thế các dấu chấm và dấu phẩy liên tiếp bằng một dấu duy nhất
    text = re.sub(r'\.+', '.', text)
    text = re.sub(r'\,+', ',', text)

    # Xóa các khoảng trắng ở đầu và cuối chuỗi
    text = text.strip()

    return text

train['full_text'] = train['full_text'].apply(clean_text)
test['full_text'] = test['full_text'].apply(clean_text)

### **4. Feature engineering**

#### **Rút trích đặc trưng đoạn văn**

Phân tích về đặc trưng đoạn văn giúp hiểu rõ hơn về cấu trúc văn bản, độ phức tạp và ngữ cảnh văn bản.
- Độ dài đoạn văn phản ánh mức độ chi tiết và độ phức tạp của văn bản.
- Số câu trong đoạn văn giúp xác định mức độ chi tiết và cách trình bày của văn bản.
- Số từ trong đoạn văn là một chỉ số quan trọng để đo lường độ phức tạp và độ dài của văn bản.
- Lấy giá trị đầu tiên và cuối cùng của mỗi đặc trưng  giúp hiểu thêm về sự biến đổi và xu hướng của các đoạn văn trong văn bản.


In [15]:
def extract_paragraph_features(data):
    # Step 1: Process data to extract paragraphs
    data['paragraph'] = data['full_text'].str.split('\n\n')

    # Step 2: Preprocess paragraphs
    data = data.explode('paragraph')
    data = data[data['paragraph'].str.strip() != ""]
    data['paragraph'] = data['paragraph'].apply(clean_text)
    data['paragraph_length'] = data['paragraph'].apply(len)
    data['sentence_count'] = data['paragraph'].apply(lambda x: len(x.split('.')))
    data['word_count'] = data['paragraph'].apply(lambda x: len(x.split()))
    data['fullstop_ratio'] = data.apply(lambda row: row['paragraph'].count('.') / len(row['paragraph']) if len(row['paragraph']) > 0 else 0, axis=1)

    # Step 3: Calculate features
    features = ['paragraph_length', 'sentence_count', 'word_count']

    # Group by essay_id and calculate the necessary aggregations
    def calculate_aggregations(group):
        aggs = {}
        for feat in features:
            aggs[f'{feat}_max'] = group[feat].max()
            aggs[f'{feat}_mean'] = group[feat].mean()
            aggs[f'{feat}_min'] = group[feat].min()
            aggs[f'{feat}_first'] = group[feat].iloc[0]
            aggs[f'{feat}_last'] = group[feat].iloc[-1]
            aggs[f'{feat}_sum'] = group[feat].sum()
            aggs[f'{feat}_q1'] = group[feat].quantile(0.25)
            aggs[f'{feat}_q3'] = group[feat].quantile(0.75)

        length_counts = {f'length_ge_{i}_count': (group['paragraph_length'] >= i).sum() for i in [50, 75, 100, 125, 150, 175, 200, 250, 300, 350, 400, 500, 600, 700]}
        length_counts.update({f'length_le_{i}_count': (group['paragraph_length'] <= i).sum() for i in [25, 49]})

        aggs.update(length_counts)

        return pd.Series(aggs)

    data = data.groupby('essay_id').apply(calculate_aggregations).reset_index()

    return data

# Sử dụng hàm để xử lý dữ liệu train và test
train_feats = extract_paragraph_features(train)
test_feats = extract_paragraph_features(test)

# Đếm số đặc trưng
feature_names = [col for col in train_feats.columns if col not in ['essay_id', 'score']]
print('Feature count in train set: ', len(feature_names))

feature_names = [col for col in test_feats.columns if col not in ['essay_id', 'score']]
print('Feature count in test set: ', len(feature_names))

Feature count in train set:  40
Feature count in test set:  40


#### **Rút trích đặc trưng câu văn**

Tương tự, việc phân tích về đặc trưng câu văn cũng giúp hiểu rõ hơn về cấu trúc, độ phức tạp và ngữ cảnh văn bản.

- Độ dài câu văn phản ánh mức độ chi tiết của thông tin và độ phức tạp trong cấu trúc câu.
- Giá trị đầu tiên và cuối cùng của mỗi đặc trưng giúp hiểu thêm về sự biến đổi và xu hướng của các câu văn trong văn bản.
- Giá trị phân vị của các đặc trưng phản ánh mức độ phân tán và sự biến động trong độ dài và số lượng từ của các câu văn.

In [16]:
def extract_sentence_features(data):
    # Step 1: Preprocess sentences
    data['sentence'] = data['full_text'].apply(clean_text).str.split('.')
    data = data.explode('sentence')
    data['sentence_len'] = data['sentence'].apply(len)
    data = data[data['sentence_len'] >= 15]
    data['sentence_word_count'] = data['sentence'].apply(lambda x: len(x.split()))
    data['comma_ratio'] = data['sentence'].apply(lambda x: x.count(',') / len(x) if len(x) > 0 else 0)

    features = ['sentence_len', 'sentence_word_count']

    # Step 2: Calculate features
    def calculate_aggregations(group):
        aggs = {}
        for feat in features:
            aggs[f'{feat}_max'] = group[feat].max()
            aggs[f'{feat}_mean'] = group[feat].mean()
            aggs[f'{feat}_min'] = group[feat].min()
            aggs[f'{feat}_first'] = group[feat].iloc[0]
            aggs[f'{feat}_last'] = group[feat].iloc[-1]
            aggs[f'{feat}_sum'] = group[feat].sum()
            aggs[f'{feat}_q1'] = group[feat].quantile(0.25)
            aggs[f'{feat}_q3'] = group[feat].quantile(0.75)

        sentence_length_counts = {f'sentence_length_ge_{i}_count': (group['sentence_len'] >= i).sum() for i in [15, 50, 100, 150, 200, 250, 300]}

        aggs.update(sentence_length_counts)

        return pd.Series(aggs)

    data = data.groupby('essay_id').apply(calculate_aggregations).reset_index()

    return data

# Xử lý dữ liệu train và test
sentence_train = extract_sentence_features(train)
sentence_test = extract_sentence_features(test)

# Kết hợp các đặc trưng mới vào dữ liệu
train_feats = train_feats.merge(sentence_train, on='essay_id', how='left')
test_feats = test_feats.merge(sentence_test, on='essay_id', how='left')

# Đếm số đặc trưng
feature_names = [col for col in train_feats.columns if col not in ['essay_id', 'score']]
print('Feature count in train set: ', len(feature_names))

feature_names = [col for col in test_feats.columns if col not in ['essay_id', 'score']]
print('Feature count in test set: ', len(feature_names))

Feature count in train set:  63
Feature count in test set:  63


#### **Rút trích đặc trưng từ**

- Độ dài từ phản ánh mức độ phức tạp, tính học thuật và mức độ thông tin của từ vựng sử dụng trong văn bản.
- Giá trị lớn nhất, trung bình, độ lệch chuẩn, và các phần tư cung cấp cái nhìn tổng quát về sự phân bố độ dài từ trong văn bản.
- Số lượng lỗi chính tả cho biết về chất lượng từ vựng và khả năng biểu đạt của văn bản.

In [17]:
def extract_word_features(data):
    # Step 1: Preprocess sentences
    data['word'] = data['full_text'].apply(clean_text).str.split(' ')
    data = data.explode('word')
    data['word_len'] = data['word'].apply(len)
    data = data[data['word_len'] != 0]

    # Step 2: Calculate features
    def calculate_aggregations(group):
        aggs = {}

        feat = 'word_len'
        aggs[f'{feat}_max'] = group[feat].max()
        aggs[f'{feat}_mean'] = group[feat].mean()
        aggs[f'{feat}_min'] = group[feat].min()
        aggs[f'{feat}_first'] = group[feat].iloc[0]
        aggs[f'{feat}_last'] = group[feat].iloc[-1]
        aggs[f'{feat}_sum'] = group[feat].sum()
        aggs[f'{feat}_q1'] = group[feat].quantile(0.25)
        aggs[f'{feat}_q3'] = group[feat].quantile(0.75)

        word_length_counts = {f'word_length_ge_{i+1}_count': (group['word_len'] >= i+1).sum() for i in range(15)}

        aggs.update(word_length_counts)

        return pd.Series(aggs)

    data = data.groupby('essay_id').apply(calculate_aggregations).reset_index()

    return data

# Xử lý dữ liệu train và test
word_train = extract_word_features(train)
word_test = extract_word_features(test)

# Kết hợp các đặc trưng mới vào dữ liệu
train_feats = train_feats.merge(word_train, on='essay_id', how='left')
test_feats = test_feats.merge(word_test, on='essay_id', how='left')

# Đếm số đặc trưng
feature_names = [col for col in train_feats.columns if col not in ['essay_id', 'score']]
print('Feature count in train set: ', len(feature_names))

feature_names = [col for col in test_feats.columns if col not in ['essay_id', 'score']]
print('Feature count in test set: ', len(feature_names))

Feature count in train set:  86
Feature count in test set:  86


In [18]:
train_feats

,essay_id,paragraph_length_max,paragraph_length_mean,paragraph_length_min,paragraph_length_first,paragraph_length_last,paragraph_length_sum,paragraph_length_q1,paragraph_length_q3,sentence_count_max,...,word_length_ge_6_count,word_length_ge_7_count,word_length_ge_8_count,word_length_ge_9_count,word_length_ge_10_count,word_length_ge_11_count,word_length_ge_12_count,word_length_ge_13_count,word_length_ge_14_count,word_length_ge_15_count
0,000d118,2652.0,2652.000000,2652.0,2652.0,2652.0,2652.0,2652.00,2652.00,14.0,...,119.0,76.0,46.0,32.0,21.0,14.0,6.0,6.0,5.0,2.0
1,000fe60,500.0,332.400000,184.0,184.0,237.0,1662.0,237.00,398.00,6.0,...,65.0,43.0,32.0,19.0,8.0,5.0,4.0,2.0,0.0,0.0
2,001ab80,1083.0,765.500000,476.0,576.0,476.0,3062.0,551.00,966.00,10.0,...,135.0,103.0,73.0,55.0,41.0,21.0,16.0,12.0,7.0,3.0
3,001bdc0,978.0,535.600000,129.0,396.0,367.0,2678.0,367.00,808.00,8.0,...,174.0,132.0,88.0,54.0,37.0,21.0,11.0,4.0,2.0,2.0
4,002ba53,690.0,363.333333,17.0,19.0,17.0,2180.0,110.75,546.50,6.0,...,138.0,112.0,69.0,51.0,25.0,20.0,12.0,9.0,5.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17302,ffd378d,443.0,284.000000,136.0,443.0,136.0,852.0,204.50,358.00,5.0,...,44.0,28.0,22.0,14.0,7.0,6.0,3.0,1.0,1.0,0.0
17303,ffddf1f,940.0,553.666667,270.0,623.0,402.0,3322.0,418.25,622.25,10.0,...,194.0,144.0,101.0,52.0,38.0,15.0,8.0,4.0,2.0,2.0
17304,fff016d,442.0,374.333333,335.0,335.0,346.0,1123.0,340.50,394.00,6.0,...,45.0,28.0,14.0,7.0,6.0,3.0,1.0,1.0,1.0,1.0
17305,fffb49b,1419.0,1419.000000,1419.0,1419.0,1419.0,1419.0,1419.00,1419.00,12.0,...,97.0,75.0,48.0,31.0,12.0,9.0,4.0,0.0,0.0,0.0


#### **Phương pháp bag-of-n-gram**

Chúng ta sẽ mã hoá các cụm n-grams trong câu thành một vector có độ dài bằng số lượng các n-grams trong từ điển và đếm tần suất xuất hiện của các cụm đó. Như vậy thì mỗi cụm n-grams sẽ trở thành một chiều biểu diễn trong không gian của vector đầu ra.

In [ ]:
# Khởi tạo CountVectorizer với các tham số cụ thể
vectorizer_cnt = CountVectorizer(
            tokenizer=lambda x: x,
            preprocessor=lambda x: x,
            token_pattern=None,
            strip_accents='unicode',
            analyzer = 'word',
            ngram_range=(2,3),
            min_df=0.10,
            max_df=0.85,
)

# Áp dụng CountVectorizer lên dữ liệu huấn luyện
train_cnt = vectorizer_cnt.fit_transform([i for i in train['full_text']])
test_cnt = vectorizer_cnt.transform([i for i in test['full_text']])

# Đưa kết quả vào DataFrame
cnt_train = pd.DataFrame(train_cnt.toarray())
cnt_test = pd.DataFrame(test_cnt.toarray())

cnt_train.columns = [f'cnt_{i}' for i in range(len(cnt_train.columns))]
cnt_test.columns = [f'cnt_{i}' for i in range(len(cnt_test.columns))]
cnt_train['essay_id'] = train_feats['essay_id']
cnt_test['essay_id'] = test_feats['essay_id']

# Hợp nhất các đặc trưng mới với dữ liệu huấn luyện đã có
train_feats = train_feats.merge(cnt_train, on='essay_id', how='left')
test_feats = test_feats.merge(cnt_test, on='essay_id', how='left')

# Đếm số đặc trưng
feature_names = list(filter(lambda x: x not in ['essay_id', 'score'], train_feats.columns))
print('Feature count in train set: ', len(feature_names))

feature_names = list(filter(lambda x: x not in ['essay_id', 'score'], test_feats.columns))
print('Feature count in test set: ', len(feature_names))

#### **Phương pháp TF-IDF (Term Frequency, Inverse Document Frequency)**

"TF-IDF là viết tắt của “Term Frequency, Inverse Document Frequency” - tạm dịch “Tần suất thuật ngữ, Tần suất tài liệu nghịch đảo”. Đó là một cách để chấm điểm tầm quan trọng của các từ (hoặc \"các thuật ngữ\") dựa trên tần suất xuất hiện của chúng xuất hiện trên nhiều tài liệu dựa trên quy tắc sau:"
- Nếu một từ xuất hiện thường xuyên trong tài liệu, điều đó rất quan trọng $\Rightarrow$ cho từ này điểm cao.
- Nhưng nếu một từ xuất hiện trong nhiều tài liệu, thì đó không phải là mã định danh duy nhất $\Rightarrow$ cho từ đó điểm thấp.

Do đó, những từ phổ biến như `the` và `for` xuất hiện trong nhiều tài liệu sẽ được scaled down. Các từ xuất hiện thường xuyên trong một tài liệu sẽ được scaled up.

Với những giải thích trên, ta có công thức tính trọng số của một từ trong tài liệu trong ngữ liệu như sau:
$$w_{i,j} = tf_{i,j} \cdot idf_i = tf_{i,j} \cdot log(\frac {N}{df_i})$$

Trong đó:
- $tf_{i,j}$: Tần suất xuất hiện của i trong j
- $N$: Tổng số tài liệu
- $df_i$: Số tài liệu chứa i

Tham khảo: [An Introduction to TF-IDF using Python](https://medium.com/analytics-vidhya/an-introduction-to-tf-idf-using-python-5f9d1a343f77)

In [ ]:
# Khởi tạo TfidfVectorizer với các tham số cụ thể
vectorizer_tfidf = TfidfVectorizer(
    tokenizer=lambda x: x,
    preprocessor=lambda x: x,
    token_pattern=None,
    strip_accents='unicode',
    analyzer='word',
    ngram_range=(3, 6),
    min_df=0.05,
    max_df=0.95,
    sublinear_tf=True,
)

# Áp dụng TfidfVectorizer lên dữ liệu huấn luyện
train_tfidf = vectorizer_tfidf.fit_transform([i for i in train['full_text']])
test_tfidf = vectorizer_tfidf.transform([i for i in test['full_text']])

# Đưa kết quả vào DataFrame
tfidf_train = pd.DataFrame(train_tfidf.toarray())
tfidf_test = pd.DataFrame(test_tfidf.toarray())

# Đổi tên các cột
tfidf_train.columns = [f'tfid_{i}' for i in range(len(tfidf_train.columns))]
tfidf_test.columns = [f'tfid_{i}' for i in range(len(tfidf_test.columns))]
tfidf_train['essay_id'] = train_feats['essay_id']
tfidf_test['essay_id'] = test_feats['essay_id']

# Hợp nhất các đặc trưng mới với dữ liệu huấn luyện đã có
train_feats = train_feats.merge(tfidf_train, on='essay_id', how='left')
test_feats = test_feats.merge(tfidf_test, on='essay_id', how='left')

# Đếm số đặc trưng
feature_names = list(filter(lambda x: x not in ['essay_id', 'score'], train_feats.columns))
print('Feature count in train set: ', len(feature_names))

feature_names = list(filter(lambda x: x not in ['essay_id', 'score'], test_feats.columns))
print('Feature count in test set: ', len(feature_names))

## **B. Xây dựng mô hình**

### **1. Chuẩn bị dữ liệu**

#### **Cài đặt kiểm tra chéo dữ liệu với StratifiedKFold**

In [ ]:
K = 5
skf = StratifiedKFold(n_splits=K, shuffle=True, random_state=42)
for i, (_, val_index) in enumerate(skf.split(train_feats, train_feats['score'])):
    train_feats.loc[val_index, 'fold'] = i

#### **Feature selection**

In [ ]:
target = ['score']
drop_columns = ['essay_id', 'fold', 'full_text', 'paragraph']

#### **Định nghĩa hàm đánh giá và hàm mất mát**

In [ ]:
# Hàm tính toán QWK
def calculate_quadratic_weighted_kappa(y_true, y_pred):
    y_true_adjusted = (y_true + a).round()
    y_pred_adjusted = (y_pred + a).clip(1, 6).round()
    qwk_score = cohen_kappa_score(y_true_adjusted, y_pred_adjusted, weights="quadratic")
    return 'QWK', qwk_score, True

# Hàm mục tiêu cho QWK
def qwk_objective(y_true, y_pred):
    y_true_adjusted = y_true + a
    y_pred_adjusted = y_pred + a
    y_pred_adjusted = y_pred_adjusted.clip(1, 6)

    f = 1 / 2 * np.sum((y_pred_adjusted - y_true_adjusted) ** 2)
    g = 1 / 2 * np.sum((y_pred_adjusted - a) ** 2 + b)

    df = y_pred_adjusted - y_true_adjusted
    dg = y_pred_adjusted - a

    grad = (df / g - f * dg / g ** 2) * len(y_true_adjusted)
    hess = np.ones(len(y_true_adjusted))
    return grad, hess

# Hàm tính toán các tham số cho QWK
def calculate_qwk_parameters(y):
    mean_value = y.mean()
    variance_value = (y ** 2).mean() - mean_value ** 2
    return np.round(mean_value, 4), np.round(variance_value, 4)

### **2. Huấn luyện mô hình**

#### **Mô hình Light Gradient-Boosting Machine**


In [ ]:
lgb_models = []
predictions, actuals = [], []

# Định nghĩa các callback cho LightGBM
training_callbacks = [
    lgb.log_evaluation(period=25),
    lgb.early_stopping(stopping_rounds=75, first_metric_only=True)
]

# Huấn luyện mô hình với Cross-Validation
for fold in range(K):

    model = lgb.LGBMRegressor(
                objective = qwk_objective,
                metrics = 'None',
                learning_rate = 0.05,
                max_depth = 5,
                num_leaves = 10,
                colsample_bytree=0.3,
                reg_alpha = 0.7,
                reg_lambda = 0.1,
                n_estimators=700,
                random_state=42,
                extra_trees=True,
                class_weight='balanced',
                device='gpu',
                verbosity = - 1)

    a, b = calculate_qwk_parameters(train_feats[train_feats['fold'] != fold]['score'])
    # Tách dữ liệu huấn luyện và đánh giá cho từng fold
    X_train = train_feats[train_feats['fold'] != fold].drop(columns=drop_columns+target)
    y_train = train_feats[train_feats['fold'] != fold]['score'] - a

    X_eval = train_feats[train_feats['fold'] == fold].drop(columns=drop_columns+target)
    y_eval = train_feats[train_feats['fold'] == fold]['score'] - a

    print(f"\nTraining fold {fold} with a: {a}, b: {b}")

    # Huấn luyện mô hình
    lgb_model = model.fit(
        X_train, y_train,
        eval_names=['train', 'valid'],
        eval_set=[(X_train, y_train), (X_eval, y_eval)],
        eval_metric=calculate_quadratic_weighted_kappa,
        callbacks=training_callbacks
    )

    lgb_models.append(lgb_model)

    # Dự đoán và điều chỉnh dự đoán với giá trị a
    fold_predictions = model.predict(X_eval) + a

    actuals.extend(y_eval)
    predictions.extend(np.round(fold_predictions, 0))

# Tính toán điểm Kappa trọng số bậc hai (QWK) cho các dự đoán
validation_score = cohen_kappa_score(actuals, predictions, weights="quadratic")

# In ra điểm đánh giá
print(f"Validation score: {validation_score}")

#### **Mô hình Neural Network**

(in progress)

Kết luận: ...

### **3. Xác định ngưỡng để làm tròn giá trị dự đoán**
Sử dụng mô hình hồi quy giúp cho việc học dữ liệu không bị overfitting. Vì thế vấn đề xảy ra khi sử dụng bài toán hồi quy cho biến dự đoán phân loại có thứ tự là xác định đúng được làm tròn số ở ngưỡng nào. **Ví dụ, giá trị dự đoán là 1.6
, vậy ta sẽ chọn 1 hay 2?**

Ở bài toán này, chúng ta có 6 labels từ 1 đến 6, cho nên chúng ta sẽ có 5 ngưỡng cần xác định để làm tròn số. Chúng ta sử dụng giá trị **quadratic weighted kappa** để làm độ đo tìm ra ngưỡng làm tròn.

In [ ]:
def find_thresholds(true, pred, interrupt=50):
    # Khởi tạo ngưỡng ban đầu
    threshold = [1.5, 2.5, 3.5, 4.5, 5.5]
    pred_round = pd.cut(pred, [-np.inf] + threshold + [np.inf],
                        labels=[1,2,3,4,5,6]).astype('int32')
    best = cohen_kappa_score(true, pred_round, weights="quadratic")

    # Tìm lần lượt 5 ngưỡng
    for k in range(5):
        for sign in [1,-1]:
            v = threshold[k]
            threshold_test = threshold.copy()
            stop = 0
            while stop < interrupt:
                # Tìm giá trị ngưỡng mới
                v += sign * 0.001
                threshold_test[k] = v
                pred_round = pd.cut(pred, [-np.inf] + threshold_test + [np.inf],
                                    labels=[1,2,3,4,5,6]).astype('int32')
                metric = cohen_kappa_score(true, pred_round, weights="quadratic")

                # Phát hiện dừng sớm vòng lặp
                if metric <= best:
                    stop += 1
                else:
                    stop = 0
                    best = metric
                    threshold = threshold_test.copy()

    # Trả kết quả ngưỡng tốt nhất
    pred_round = pd.cut(pred, [-np.inf] + threshold + [np.inf],
                        labels=[1,2,3,4,5,6]).astype('int32')

    return threshold, cohen_kappa_score(true, pred_round, weights="quadratic")

In [ ]:
thresholds, best_score = find_thresholds(train.score.values, predictions, 100)
print("Ngưỡng làm tròn tốt nhất là:", thresholds)
print("Với điểm quadratic weighted kappa:", best_score)


### **4. Dự đoán kết quả**

In [ ]:
# Danh sách để lưu trữ dự đoán từ các mô hình
test_predictions = []
drop_columns = ['essay_id', 'full_text', 'paragraph']

# Lặp qua từng mô hình để dự đoán trên tập dữ liệu kiểm thử
for fold, model in enumerate(lgb_models):
    X_test = test_feats.drop(columns=drop_columns)
    fold_predictions = model.predict(X_test) + a
    test_predictions.append(fold_predictions)

# Kết hợp kết quả từ các mô hình
for i, fold_predictions in enumerate(test_predictions):
    test_feats[f"score_pred_{i}"] = fold_predictions

# Tính toán giá trị dự đoán trung bình và làm tròn kết quả để ra kết quả cuối cùng
test_feats["score"] = test_feats[[f"score_pred_{fold}" for fold in range(K)]].mean(axis=1)
test_feats["score"] = pd.cut(test_feats["score"], [-np.inf] + thresholds + [np.inf], labels=[1,2,3,4,5,6]).astype('int32')


# In ra các giá trị dự đoán
test_feats[['essay_id', 'score']].head()